In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

* This notebook to analyse the No of Beds availability in Hosipital and check wether it is possible to accomadate More Covid-19 cases based on the Population census2011

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df_Hospital =  pd.read_csv("/kaggle/input/covid19-in-india/HospitalBedsIndia.csv")
df_Hospital.head()

In [ ]:
df_population = pd.read_csv("/kaggle/input/covid19-in-india/population_india_census2011.csv")
df_population.head()


# **Data Understanding and cleansing **

In [ ]:
# Check the info of the Hospital Dataset
df_Hospital.info()

In [ ]:
df_Hospital.shape   #38 rows and 12 columns

In [ ]:
# We have to convert datatypes of some object columns into Numeric.It is required to do data analysis
df_Hospital_cols_obj=df_Hospital.select_dtypes(include = ['object']).columns.to_list()
df_Hospital_cols_obj=df_Hospital_cols_obj[1:]   # removing state columns

# apply datatype function

for i in df_Hospital_cols_obj:
    df_Hospital[i]=df_Hospital[i].str.replace(',', '').apply(pd.to_numeric)

# Check the datatype
df_Hospital.info()



In [ ]:
# Null check for Hospital Dataset
df_Hospital.isnull().sum()

In [ ]:
# 'Unnamed' column is not required for data analysis. Hence we can remove it from dataset
df_Hospital=df_Hospital.drop(['Unnamed: 12','Unnamed: 13' ],axis=1)
round(df_Hospital.isnull().sum()/len(df_Hospital.index)*100,2)

In [ ]:
df_Hospital.loc[df_Hospital.isnull().sum(axis=1)>4,:]

df_Hospital=df_Hospital.loc[df_Hospital.isnull().sum(axis=1)<=4,:]
print(df_Hospital.shape)
round(df_Hospital.isnull().sum()/len(df_Hospital.index)*100,2)


We can impute 'NumSubDistrictHospitals_HMIS' values with 0 as it doesn't add any value if we do it with Mean or median. This is belong to number of hospitals for each state or union territory. Hence we can impute with 0 instead of removing the rows or columns

In [ ]:
df_Hospital.loc[pd.isnull(df_Hospital['NumSubDistrictHospitals_HMIS']),'NumSubDistrictHospitals_HMIS']=0
round(df_Hospital.isnull().sum()/len(df_Hospital.index)*100,2)

Now Hospital Data set is ready for data analysis and will do same process for Population dataset as well

In [ ]:
df_population.info()

In [ ]:
# Population- Null check
round(df_population.isnull().sum()/len(df_Hospital.index)*100,2)

Now both Population and hospital data set is available and will merge into as master dataset for data analysis

In [ ]:
# Merging Population and hospitals


df_population.rename(columns={"State / Union Territory": "State/UT"},inplace=True)
# df_population.head()
df_master = df_population.merge(df_Hospital, on="State/UT" ,how="inner")
df_master.head()



In [ ]:
round(df_master.isnull().sum()/len(df_master.index)*100,2)

# Data Preparation

Now we need to derive some useful columns to get more insights

In [ ]:
# Removing string character from Area and Density to convert into float data type
df_master['Area']=df_master['Area'].apply(lambda x : x[:x.index('k')])
df_master['Density']=df_master['Density'].apply(lambda x : x[:x.index('/')])




In [ ]:
# conver to int
df_master['Area']=df_master['Area'].str.replace(",", "").apply( lambda x : int(x))
df_master['Density']=df_master['Density'].str.replace(",", "").apply( lambda x : int(x))
df_master.head()

In [ ]:
# Deriving Rural and Urban Density for each state

df_master['Rural population_Density'] =   df_master['Rural population'] /df_master['Area']
df_master['Urban population_Density'] =   df_master['Urban population'] /df_master['Area']
df_master.head()

In [ ]:
df_m=df_master.drop(['Sno_x','Sno_y'],axis=1)
df_corr=df_m.corr()
fig_dims = (20, 20)
fig, ax = plt.subplots(figsize=fig_dims)
sns.heatmap(df_corr,ax=ax,annot=True)

In [ ]:
# Lets get the insights on South India  - Tamil Nadu,Andhra Pradesh, Kerala, Karnataka,Puducherry

In [ ]:
a=['Tamil Nadu','Andhra Pradesh', 'Kerala', 'Karnataka','Puducherry']
df_south = df_master.loc[df_master['State/UT'].isin (a)]
df_south

In [ ]:
sns.barplot(x="State/UT", y='NumRuralHospitals_NHP18', data=df_south)

In [ ]:
sns.barplot(x="State/UT", y='NumUrbanHospitals_NHP18', data=df_south)

In [ ]:
sns.barplot(x="State/UT", y='Population', data=df_south)

Karnataka and Kerala have more Urban and Rural beds in hospitals than other south states.

Lets try to find no of beds per 1000 person in rural and Urban area for south states

#  No of rural and Urban beds per 1000 population

It is easy intepret the ratio of bed in every 1000 person on rural and urban area by statewise. This will enable the Govt to take appropriate action on state wise

In [ ]:
# calacluate the no of rural and Urban beds per 1000 population
df_south['no_of_rural_beds_per_1000_population'] = df_south['NumRuralBeds_NHP18']/(df_south['Rural population']/1000)
df_south['no_of_urban_beds_per_1000_population'] = df_south['NumUrbanBeds_NHP18']/(df_south['Urban population']/1000)


In [ ]:
df_south

In [ ]:
sns.barplot(x="State/UT", y='no_of_rural_beds_per_1000_population', data=df_south)

# plt.show()

Tamil Nadu has around 1 rural bed per 1000 person in the state.Andhra Pradesh has least rural bed per 1000 person which is 0.2. But as overall, all the south states should increase the bed to combat the Covid-19 Disease

In [ ]:
sns.barplot(x="State/UT", y='no_of_urban_beds_per_1000_population', data=df_south)

Puducherry has around 4 rural bed per 1000 person in the state.Tamil Nadu has least rural bed per 1000 person which is 1. But as overall, all the south states should increase the bed to combat the Covid-19 Disease

As Part of beds per 1000 person analysis statewise,Tamil Nadu maintain same ratio (~1 bed in 1000 person) in both Rural and Urban areas.

Of course it is difficult to accomodate more patients in situation like Covid-19. Hence Govt of India should take immediate action to increase the bed count in statewise